In [50]:
# import some helpful python packages 
import numpy as np

from astropy.table import Table
from astropy.convolution import convolve, Gaussian1DKernel

import matplotlib 
import matplotlib.pyplot as plt

In [51]:
from dl import queryClient as qc

In [52]:
print(qc.schema('desi_edr'))


Schema: desi_edr

      Table Name   Description
      ----------   -----------
        exposure   Summary quantities for every DESI exposure
     fiberassign   Quantities obtained when a DESI target is assigned to a fiber
           frame   Summary quantities for each petal of the DESI instrument i
                   n a given exposure; in normal operation there are ten fram
                   es for every exposure
      photometry   Photometric quantities from LS DR9 for every TARGETID
       potential   For a given tile, this table lists all targets that could 
                   have received a fiber assignment
          target   The quantities obtained when photometric objects are analy
                   zed in the target selection process
            tile   Summary quantities for every DESI tile (pointing on the sky)
x1p5__zpix__allwise__source   desi_edr.zpix and xmatch_std.allwise__source crossmatch (1
                   .5 arcsec)
x1p5__zpix__gaia_dr3__gaia_source   desi_edr

In [53]:
print(qc.schema('desi_edr.photometry'))


Schema: desi_edr
 Table: photometry

     Column Name   Description
     -----------   -----------
              ra   Right ascension
             dec   Declination
            elon   Ecliptic longitude
            elat   Ecliptic latitude
            glon   Galactic longitude
            glat   Galactic latitude
           ls_id   Unique Legacy Survey object ID
          ref_id   Tyc1*1000000+Tyc2*10+Tyc3 for Tycho-2; sourceid for Gaia DR2
        targetid   Unique DESI target ID
         ra_ivar   Right ascension inverse variance
        dec_ivar   Declination inverse variance
      dchisq_psf   Difference in chi-squared between PSF (stellar) Tractor mo
                   del fits
      dchisq_rex   Difference in chi-squared between REX (round exponential g
                   alaxy) Tractor model fits
      dchisq_dev   Difference in chi-squared between DEV (deVauc) Tractor mod
                   el fits
      dchisq_exp   Difference in chi-squared between EXP (exponential) Tracto
 

In [54]:
query = """
SELECT ph.flux_g, ph.targetid, ph.ls_id, ph.ref_id
FROM desi_edr.photometry AS ph
WHERE ph.targetid = 39633318391581576
LIMIT 10
""" 
zpix = qc.query(sql = query, fmt = 'table')
print(zpix)

  flux_g       targetid          ls_id       ref_id
--------- ----------------- ---------------- ------
262.54828 39633318391581576 9907738672892808 561686


In [55]:
file_path = "/home/jeremiasrodriguez/data/astroclip_desi.1.1.5.h5"

import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

# Open the HDF5 file
all_target_ids = []

with h5py.File(file_path, 'r') as h5_file:
    # Loop through each group in the file and concatenate redshifts
    for group_name in h5_file.keys():
        targetid_data = h5_file[f"{group_name}/targetids"][:]
        all_target_ids.append(targetid_data)

# Combine all redshifts into a single array
combined_targetids = np.concatenate(all_target_ids)

In [56]:
# Split the combined_targetids array into smaller chunks (e.g., 1000 ids per chunk)
chunk_size = 100
chunks = [combined_targetids[i:i + chunk_size] for i in range(0, len(combined_targetids), chunk_size)]

all_results = []

# Iterate over chunks
for idx, chunk in enumerate(chunks):
    print("Going over chunk ", idx, "/", len(chunks))
    # Convert the chunk into a string that can be used in SQL
    ids_str = ', '.join([str(i) for i in chunk])

    # Use the CTE with the targetid values
    query = f"""
    WITH target_ids (targetid) AS (
        VALUES {', '.join(f'({i})' for i in chunk)}
    )
    SELECT ph.flux_g, ph.flux_r, ph.flux_z, ph.flux_ivar_g, ph.flux_ivar_r, ph.flux_ivar_z, ph.targetid, tr.mag_g, tr.mag_r, tr.mag_z, tr.flux_g, tr.dered_mag_g, tr.dered_mag_r, tr.dered_mag_z
    FROM desi_edr.photometry AS ph INNER JOIN ls_dr9.tractor as tr ON ph.ls_id = tr.ls_id
    JOIN target_ids AS t
    ON ph.targetid = t.targetid
    """
    
    # Execute the query for this chunk
    zpix = qc.query(sql=query, fmt='table')
    
    # Convert the result to a Pandas DataFrame
    df = zpix.to_pandas()

    # Store the DataFrame result from each chunk
    all_results.append(df)

# Concatenate all DataFrames from chunks
final_results = pd.concat(all_results, ignore_index=True)

Going over chunk  0 / 1980
Going over chunk  1 / 1980
Going over chunk  2 / 1980
Going over chunk  3 / 1980
Going over chunk  4 / 1980
Going over chunk  5 / 1980
Going over chunk  6 / 1980
Going over chunk  7 / 1980
Going over chunk  8 / 1980
Going over chunk  9 / 1980
Going over chunk  10 / 1980
Going over chunk  11 / 1980
Going over chunk  12 / 1980
Going over chunk  13 / 1980
Going over chunk  14 / 1980
Going over chunk  15 / 1980
Going over chunk  16 / 1980
Going over chunk  17 / 1980
Going over chunk  18 / 1980
Going over chunk  19 / 1980
Going over chunk  20 / 1980
Going over chunk  21 / 1980
Going over chunk  22 / 1980
Going over chunk  23 / 1980
Going over chunk  24 / 1980
Going over chunk  25 / 1980
Going over chunk  26 / 1980
Going over chunk  27 / 1980
Going over chunk  28 / 1980
Going over chunk  29 / 1980
Going over chunk  30 / 1980
Going over chunk  31 / 1980
Going over chunk  32 / 1980
Going over chunk  33 / 1980
Going over chunk  34 / 1980
Going over chunk  35 / 1980
Go

In [59]:
final_results

,flux_g,flux_r,flux_z,flux_ivar_g,flux_ivar_r,flux_ivar_z,targetid,mag_g,mag_r,mag_z,flux_g_1,dered_mag_g,dered_mag_r,dered_mag_z
0,33.110810,72.423780,132.092620,102.041565,17.031120,10.321765,39633062060887564,18.700075,17.850298,17.197803,33.110810,18.508340,17.721142,17.125560
1,371.072240,845.922060,1490.594200,13.769390,2.023977,1.518554,39633325282823017,16.076353,15.181674,14.566602,371.072240,16.035305,15.154023,14.551135
2,18.589428,53.638756,100.641020,82.780760,22.142246,10.029376,39632996176760219,19.326836,18.176304,17.493063,18.589428,19.287600,18.149874,17.478280
3,13.676310,57.358240,152.692860,40.324482,18.190311,8.479033,39633275198636473,19.660078,18.103510,17.040453,13.676310,19.630505,18.083590,17.029310
4,21.319311,57.717460,111.225586,99.228850,27.969105,14.979363,39633136551725040,19.178066,18.096731,17.384490,21.319311,19.133085,18.066431,17.367540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197971,1.455048,7.193043,21.717264,287.985720,79.154690,27.207714,39633301253652804,22.092806,20.357718,19.157988,1.455048,22.061825,20.336847,19.146313
197972,1.966346,6.495502,15.439288,549.103400,128.192120,58.098927,39633251840558896,21.765850,20.468468,19.528430,1.966346,21.719648,20.437346,19.511024
197973,1.051315,6.575119,32.949200,38.295680,11.294885,7.582934,39633136459448434,22.445667,20.455240,18.705387,1.051315,22.385256,20.414547,18.682625
197974,2.038499,7.177494,23.432950,68.088520,21.225655,5.722333,39633158366298541,21.726725,20.360067,19.075432,2.038499,21.697094,20.340109,19.064268


In [60]:
# Save the final_results to an HDF5 file
final_results.to_hdf('desi_edr_fluxes.h5', key='df', mode='w')

In [61]:
print(qc.schema('ls_dr10'))


Schema: ls_dr10

      Table Name   Description
      ----------   -----------
          apflux   Aperture flux values of tractor objects (combined DR10 sou
                   thern region and DR9 northern region)
        apflux_s   Aperture flux values of tractor objects (DR10 southern region)
          bricks   Table with RA, Dec bounds of each geometrical brick on the sky
        bricks_s   Table with RA, Dec bounds of each geometrical brick on the
                   sky (DR10 southern region)
 depth_summary_s   A concatenation of the depth histograms for each brick (DR
                   10 southern region)
         photo_z   Photometric redshifts
         tractor   Main tractor photometry catalog (combined DR10 southern re
                   gion and DR9 northern region); Less frequently used column
                   s placed in apflux and wise tables
       tractor_s   Main tractor photometry catalog (DR10 southern region); Le
                   ss frequently used columns place

In [66]:
print(qc.schema('ls_dr10.photo_z'))
# FROM HERE, I THINK WE CARE ABOUT THE z_spec - WE SHOULD VALIDATE IT MATCHES THE ASTROCLIP REPORTED REDSHIFT


Schema: ls_dr10
 Table: photo_z

     Column Name   Description
     -----------   -----------
           ls_id   Unique LS object ID
         brickid   Brick ID [1,662174]
           objid   Catalog object number within this brick; a unique identifi
                   er hash is release,brickid,objid; objid spans [0,N-1] and
                   is contiguously enumerated within each brick
     z_phot_mean   photo-z derived from the mean of the photo-z PDF
   z_phot_median   photo-z derived from the median of the photo-z PDF
      z_phot_std   Standard deviation of the photo-zs derived from the photo-z PDF
      z_phot_l68   Lower bound of the 68% confidence region, derived from the
                   photo-z PDF
      z_phot_u68   Upper bound of the 68% confidence region, derived from the
                   photo-z PDF
      z_phot_l95   Lower bound of the 95% confidence region, derived from the
                   photo-z PDF
      z_phot_u95   Upper bound of the 95% confidence region

In [68]:
print(qc.schema('ls_dr10.tractor_s'))
'''
Interesting:
- ref_id Could be the ID
- flug_g, flux_r,flux_z
- flux_ivar_g, ...
- ls_id Could be the ID
- mag_g, mag_r, mag_z
- dered_flux_{}  and dered_mag_{}
'''


Schema: ls_dr10
 Table: tractor_s

     Column Name   Description
     -----------   -----------
    dered_mag_w1   Dereddened w1-band magnitude
    dered_mag_w2   Dereddened w2-band magnitude
    dered_mag_w3   Dereddened w3-band magnitude
         release   Integer denoting the camera and filter set used, which wil
                   l be unique for a given processing run of the data (as doc
                   umented here_)
         brickid   Brick ID [1,662174]
       brickname   Name of brick, encoding the brick sky position, eg 1126p22
                   2 near RA=112.6, Dec=+22.2
           objid   Catalog object number within this brick; a unique identifi
                   er hash is release,brickid,objid;  objid spans [0,N-1] and
                   is contiguously enumerated within each brick
   brick_primary   True if the object is within the brick boundary
        maskbits   Bitwise mask indicating that an object touches a pixel in 
                   the coadd/*/*/*maskbi

In [19]:
from dl import authClient as ac
from dl import storeClient as sc
from dl import queryClient as qc
from dl.helpers.utils import convert        # to use Pandas
import os
from getpass import getpass

do_cleanup = True                          # Remove any files/tables created by this notebook

#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()
print("Listing mydb tables:\n" + qc.mydb_list())

Listing mydb tables:
No tables


In [27]:
print(qc.schema('desi_edr.x1p5__zpix__allwise__source'))


Schema: desi_edr
 Table: x1p5__zpix__allwise__source

     Column Name   Description
     -----------   -----------
             id1   ID in left/first table (id)
             ra1   Right ascension in left/first table (mean_fiber_ra)
            dec1   Declination in left/first table (mean_fiber_dec)
             id2   ID in right/second table (source_id)
             ra2   Right ascension in right/second table (ra)
            dec2   Declination in right/second table (dec)
        distance   Distance between ra1,dec1 and ra2,dec2 (arcsec)



In [40]:
print(qc.schema('ls_dr9.tractor'))


Schema: ls_dr9
 Table: tractor

     Column Name   Description
     -----------   -----------
    dered_mag_w2   Dereddened w2-band magnitude
    dered_mag_w3   Dereddened w3-band magnitude
           ls_id   Unique LS object ID
             dec   Declination at equinox J2000
              ra   Right ascension at equinox J2000
            elat   Ecliptic Latitude
            elon   Ecliptic Longitude
            glat   Galactic Latitude
            glon   Galactic Longitude
         mjd_max   Maximum Modified Julian Date of observations used to const
                   ruct the model of this object
         mjd_min   Minimum Modified Julian Date of observations used to const
                   ruct the model of this object
          ref_id   Reference catalog identifier for this star; Tyc1*1,000,000
                   +Tyc2*10+Tyc3 for Tycho2; sourceid for Gaia DR2 and SGA
         brickid   Brick ID [1,662174]
      blob_nea_g   Blob-masked noise equivalent area in g.
      blob_nea_

In [39]:
query = """
SELECT count(ph.ls_id)
FROM desi_edr.photometry as ph INNER JOIN ls_dr9.tractor as tr ON ph.ls_id = tr.ls_id
""" 
zpix = qc.query(sql = query, fmt = 'table')
print(zpix)

 count 
-------
6031250
